In [12]:
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt

In [9]:
# load and show file
images_folder_path = os.getcwd() + "\images\\" 

image_path = images_folder_path + "key1.jpg"

image = cv2.imread(image_path)
cv2.imshow("orginal", image)
cv2.waitKey(0)
cv2.destroyAllWindows()
image_shape = image.shape
print(image.shape)

(2448, 3264, 3)


In [10]:
# scale image down to 1000 pixels
r = 1000.0/image.shape[1]
dim = (1000,int(image.shape[0] * r))
resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
cv2.imshow("resized", resized)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [24]:
# reduce reflections in image

hsv_image = cv2.cvtColor(resized, cv2.COLOR_RGB2HSV)
h, s, v = cv2.split(hsv_image)

clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
v = clahe.apply(v)

hsv_image = cv2.merge([h, s, v])
hsv_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2RGB)
cv2.imshow("reduced reflection", hsv_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [33]:
# reduce shadow and background

# DID NOT WORK

# fgbg = cv2.createBackgroundSubtractorMOG2()

fgmask_image = fgbg.apply(hsv_image)



cv2.imshow("fgmask", fgmask_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [34]:
# extract key from image
gray_image = cv2.cvtColor(hsv_image, cv2.COLOR_BGR2GRAY)
# cv2.imshow("gray image", gray)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

edges = cv2.Canny(gray_image,100,150)
cv2.imshow("edges", edges)
cv2.waitKey(0)
cv2.destroyAllWindows() 